In [1]:
from opt_einsum import contract
import torch
import numpy as np
import os,pdb
torch.set_default_tensor_type(torch.cuda.DoubleTensor)
device=torch.device('cuda:1')
torch.cuda.set_device(device)


from scipy.sparse.linalg import eigs
from linearized import mysvd, myeigh, get_linearized_cylinder,get_linearized_cylinder_np, verify_linear_operator, check_hermicity
from ScalingDimensions import get_scaling_dimensions
from HOTRGZ2 import HOTRG_layers

def _toP(t):
    if isinstance(t,list):
        return [_toP(tt) for tt in t]
    elif isinstance(t,torch.Tensor):
        return t.detach().cpu().numpy()
    else:
        return t

filename='data/transfer_matrix_X16_TNR_L10'


options={
    'tensor_path':'data/tnr_X16_tensors.pkl',
    'iLayer':10,
    'max_dim':16,
    
    'svd_max_iter':100,
    'svd_tol':1e-16,
    'svd_num_eigvecs':16,
    
    '_version':1,
}


if os.path.exists(filename+'_options.pkl'):
    _options=torch.load(filename+'_options.pkl',map_location=device)
    if not(options==_options):
        def tryRemove(filename):
            if os.path.exists(filename):
                os.remove(filename)
        tryRemove(filename+'_options.pkl')
        tryRemove(filename+'_eigs_cyl.pkl')

torch.save(options,filename+'_options.pkl')
print('file saved: ',filename+'_options.pkl')

layers,Ts,logTotals=torch.load(options['tensor_path'],map_location=device)

iLayer=options['iLayer']
T=Ts[iLayer].clone()
assert T.shape[0]==options['max_dim']

#T=T*0;T[0,0,0,0]=1

Mc1=get_linearized_cylinder(T)
Mc=get_linearized_cylinder_np(_toP(T))

u=np.random.randn(16**4)
#u=u*0;u[0]=1
v=Mc.matvec(u)
v1=Mc1.matvec(u)
print('MV, torch',v[:10])
print('MV, np',v1[:10])










/home/wenhan/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


file saved:  data/transfer_matrix_X16_TNR_L10_options.pkl
torch
v tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
T0 tensor([ 6.8351e-02, -1.0741e-19,  1.2400e-34, -2.5476e-03, -1.8271e-21,
        -1.8229e-35, -1.9170e-21,  1.8942e-17, -1.6103e-04, -9.8878e-19])
v1 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
np
v [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
T0 [ 6.83514570e-02 -1.07408111e-19  1.24002597e-34 -2.54762085e-03
 -1.82707429e-21 -1.82292003e-35 -1.91698644e-21  1.89416569e-17
 -1.61028428e-04 -9.88775926e-19]
v1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
np
v [ 0.33789105 -1.34623499  0.46291843  0.07530262 -0.20210406 -0.51766169
 -0.58405054  0.75720258  0.16361117  0.96984461]
T0 [ 6.83514570e-02 -1.07408111e-19  1.24002597e-34 -2.54762085e-03
 -1.82707429e-21 -1.82292003e-35 -1.91698644e-21  1.89416569e-17
 -1.61028428e-04 -9.88775926e-19]
v1 [-4.04202631e-06 -1.80615067e-05  1.49468692e-06 -2.21349275e-07
 -1.14228931e-06  5.84962036e-08 -8.07645651e-08  5.63888861e-09
 -6.70005154e-10

In [44]:
T=T/T.max()

In [45]:
from opt_einsum import contract
Tp=_toP(T)

In [54]:

contract('iIab,jJbc,kKcd,lLda,IJKL->ijkl',T,T,T,T,T)

tensor([[[[ 1.8304e+00,  2.9920e-05, -5.8784e-06,  ..., -4.3882e-05,
           -1.2769e-08,  3.0461e-09],
          [-3.0041e-05,  7.1263e-01,  9.9849e-02,  ..., -1.6894e-08,
           -1.9428e-05, -1.6961e-05],
          [-5.8581e-06, -9.9855e-02, -5.4456e-02,  ...,  1.1367e-08,
            3.0618e-07, -1.5607e-06],
          ...,
          [-4.4987e-05,  1.6828e-08,  1.1416e-08,  ...,  1.1791e-07,
            2.5792e-11, -2.7769e-12],
          [-1.2882e-08,  1.9195e-05,  2.0979e-07,  ...,  2.6022e-11,
            4.9000e-09,  1.4182e-08],
          [-3.0808e-09, -1.7285e-05,  1.4881e-06,  ...,  2.8358e-12,
           -1.4148e-08,  1.3689e-08]],

         [[-7.0786e-06,  5.7782e-01,  2.0132e-03,  ..., -5.5792e-09,
           -2.1925e-06, -1.9173e-05],
          [ 7.1659e-01,  1.4445e-05, -2.0363e-06,  ..., -6.4834e-05,
           -4.8528e-09,  3.6819e-09],
          [ 9.8897e-02,  2.4703e-06, -1.2378e-08,  ...,  2.7979e-05,
            6.2157e-10, -2.2285e-09],
          ...,
     

In [50]:

contract('iIab,jJbc,kKcd,lLda,IJKL->ijkl',Tp,Tp,Tp,Tp,Tp)

array([[[[ 4.13017391e+00, -1.36689856e-04, -1.00463733e-04, ...,
          -1.18516877e-03, -1.83049591e-07,  6.86131938e-08],
         [-2.70955408e-04,  5.07847132e+00,  5.93060045e-02, ...,
           4.62072339e-08,  5.20869083e-04, -3.29287814e-04],
         [-3.15380791e-06,  7.13223090e-02, -2.54755787e-01, ...,
           1.85303732e-09, -1.50271059e-04,  6.11980257e-05],
         ...,
         [-1.25070644e-03,  1.38014955e-07,  1.67215073e-07, ...,
           2.68064386e-07,  3.12500414e-10,  8.82869295e-12],
         [-1.71685434e-07, -6.31209410e-04, -1.81993850e-04, ...,
           4.79427553e-10,  1.73941771e-07,  2.92815340e-07],
         [-6.74703300e-08, -4.95554661e-04, -1.39886313e-04, ...,
           1.19940092e-10, -2.36696765e-07,  1.30315062e-07]],

        [[-1.13094183e-04,  4.97826379e+00,  5.02254515e-01, ...,
          -9.86676670e-09, -4.00169123e-04, -1.97612744e-04],
         [ 5.71755797e+00,  1.17758928e-04, -3.07642353e-05, ...,
          -5.58408038e

In [59]:
np.save('nperr.npy',Tp)

In [13]:
T=T*0;T[0,0,0,0]=1

Mc1=get_linearized_cylinder(T)
Mc=get_linearized_cylinder_np(_toP(T))

u=np.random.randn(16**4)
u=u*0;u[0]=1
v=Mc.matvec(u)
v1=Mc1.matvec(u)
print('MV, torch',v[:10])
print('MV, np',v1[:10])

torch
v tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
T0 tensor([1., -0., 0., -0., -0., -0., -0., 0., -0., -0.])
v1 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
np
v [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
T0 [ 1. -0.  0. -0. -0. -0. -0.  0. -0. -0.]
v1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
np
v [ 1.  0. -0.  0. -0. -0.  0. -0.  0.  0.]
T0 [ 1. -0.  0. -0. -0. -0. -0.  0. -0. -0.]
v1 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
torch
v tensor([1., 0., -0., 0., -0., -0., 0., -0., 0., 0.])
T0 tensor([1., -0., 0., -0., -0., -0., -0., 0., -0., -0.])
v1 tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
MV, torch [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
MV, np [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [2]:
from opt_einsum import contract
import numpy as np
import torch

Tp=np.load('nperr.npy')
Tt=torch.tensor(Tp)



In [3]:

contract('iIab,jJbc,kKcd,lLda,IJKL->ijkl',Tp,Tp,Tp,Tp,Tp)

array([[[[ 3.71195359e+00,  1.66246855e-04, -7.04846884e-06, ...,
          -5.20841817e-04, -5.88884845e-08, -2.61933268e-08],
         [-2.58925911e-06,  1.89939990e+00,  5.69195922e-02, ...,
          -1.37574894e-07,  1.41861271e-04, -1.18692140e-04],
         [-3.22640399e-05, -7.72482191e-02, -9.39880176e-02, ...,
           9.61592214e-08, -3.02435168e-05,  3.60545276e-05],
         ...,
         [-2.42605069e-04, -4.25039907e-08,  1.00901807e-08, ...,
           1.70791067e-07,  1.33702448e-10, -2.92370053e-11],
         [-8.63527533e-08, -5.21647031e-05, -1.19275369e-05, ...,
           3.40228155e-10,  4.13080579e-08,  1.83030730e-07],
         [-3.06684950e-08, -7.40169983e-05, -3.52981170e-06, ...,
           7.49945945e-11, -6.94416038e-08,  8.41938764e-08]],

        [[ 3.42371239e-05,  3.59454194e+00, -3.76557310e-01, ...,
          -4.86907806e-08,  3.24114825e-04, -1.58115630e-04],
         [ 2.94166956e+00,  1.60183688e-04, -4.15802231e-06, ...,
          -5.70334950e

In [4]:

contract('iIab,jJbc,kKcd,lLda,IJKL->ijkl',Tt,Tt,Tt,Tt,Tt)

tensor([[[[ 1.8304e+00,  2.9920e-05, -5.8784e-06,  ..., -4.3882e-05,
           -1.2769e-08,  3.0461e-09],
          [-3.0041e-05,  7.1263e-01,  9.9849e-02,  ..., -1.6894e-08,
           -1.9428e-05, -1.6961e-05],
          [-5.8581e-06, -9.9855e-02, -5.4456e-02,  ...,  1.1367e-08,
            3.0618e-07, -1.5607e-06],
          ...,
          [-4.4987e-05,  1.6828e-08,  1.1416e-08,  ...,  1.1791e-07,
            2.5792e-11, -2.7769e-12],
          [-1.2882e-08,  1.9195e-05,  2.0979e-07,  ...,  2.6022e-11,
            4.9000e-09,  1.4182e-08],
          [-3.0808e-09, -1.7285e-05,  1.4881e-06,  ...,  2.8358e-12,
           -1.4148e-08,  1.3689e-08]],

         [[-7.0786e-06,  5.7782e-01,  2.0132e-03,  ..., -5.5792e-09,
           -2.1925e-06, -1.9173e-05],
          [ 7.1659e-01,  1.4445e-05, -2.0363e-06,  ..., -6.4834e-05,
           -4.8528e-09,  3.6819e-09],
          [ 9.8897e-02,  2.4703e-06, -1.2378e-08,  ...,  2.7979e-05,
            6.2157e-10, -2.2285e-09],
          ...,
     